In [1]:
import pandas as pd
import numpy as np

In [2]:
df_bone_content = pd.read_csv("Bone_Conetnt.csv")
df_bone_content

,Type_of_Bone,Bone_percentage
0,Chicken Neck,0.75
1,Chicken Feet,0.60
2,Chicken Wing,0.46
3,Duck Neck,0.50
4,Duck Feet,0.60
5,Whole Quail,0.10
6,Cornish Hen,0.40
7,Rabbit Head,0.75
8,Rabbit front leg,0.15
9,Rabbit hind leg,0.14


In [56]:
# functions
def calculate_meal_ratio(ingredients):
    ratio = {}
    total_weight = np.sum([items for key, items in ingredients.items()])
    ratio['liver'] = ingredients['liver']/total_weight
    ratio['other secreting organ'] = ingredients['other secreting organ']/total_weight
    ratio['muscle organ'] = ingredients['muscle organ']/total_weight
    meat_on_bone = 0
    bone_weight = 0
    for bone in ingredients.keys() - ['liver','other secreting organ','muscle organ','muscle meat']:
        bone_on_bone = ingredients[bone]*\
                        (df_bone_content.loc[df_bone_content.Type_of_Bone.str.lower() == bone.lower(), 'Bone_percentage'].values[0])
        bone_weight+=bone_on_bone
        meat_on_bone+= ingredients[bone]-bone_on_bone
    ratio['muscle meat'] = (ingredients['muscle meat']+meat_on_bone)/total_weight
    ratio['bone'] = bone_weight/total_weight
    ratio = pd.DataFrame.from_dict(ratio,orient='index', columns=['percent'])
    return ratio

puppy_barf_model = {'muscle meat':0.52,
                    'muscle organ':0.08,
                   'bone':0.15,
                   'liver':0.07,
                   'other secreting organ':0.07,
                   'veggie':0.07,
                   'nut':0.03,
                   'fruit':0.01}

adult_barf_model = {'muscle meat':0.61,
                    'muscle organ':0.09,
                   'bone':0.10,
                   'liver':0.05,
                   'other secreting organ':0.05,
                   'veggie':0.07,
                   'nut':0.02,
                   'fruit':0.01}

def calculate_meat_to_bone_ratio(meat_bone,model):
    ratio = {}
    total_weight = np.sum([items for key, items in meat_bone.items()])
    meat_weight = meat_bone['muscle meat']
    bone_weight = 0
    for bone in meat_bone.keys() - ['muscle meat']:
        bone_on_bone = meat_bone[bone]*\
                        (df_bone_content.loc[df_bone_content.Type_of_Bone.str.lower() == bone.lower(), 'Bone_percentage'].values[0])
        bone_weight+=bone_on_bone
        meat_weight+= meat_bone[bone]-bone_on_bone
    ratio['muscle meat'] = meat_weight/total_weight
    ratio['bone'] = bone_weight/total_weight
    ratio = pd.DataFrame.from_dict(ratio,orient='index', columns=['relevant_ratio'])
    ideal_total = model['muscle meat'] + model['bone']
    ratio['ideal_relevant_ratio'] = [model['muscle meat']/ideal_total, model['bone']/ideal_total]
    return ratio

def calculate_organ_per_serving(serving, model):
    ingredients = {}
    ingredients['liver'] = serving * model['liver']
    ingredients['other_organ'] = serving * model['other secreting organ']
    ingredients['muscle_organ'] = serving * model['muscle organ']
    return ingredients
    
def calculate_meatbone_per_serving(serving, model):
    return serving * model['muscle meat'] + serving * model['bone']
    
def calculate_veggie_fruit_nut_per_serving(serving, model):
    ingredients = {}
    ingredients['veggie'] = serving * model['veggie']
    ingredients['fruit'] = serving * model['fruit']
    ingredients['nut'] = serving * model['nut']
    return ingredients

def meal_prep(daily_serving, n_days, model, bone_type):
#     ingredients = {}
    ratio = {}
    total_weight = daily_serving * n_days
    vfn_serving = calculate_veggie_fruit_nut_per_serving(total_weight, model)
    organ_serving = calculate_organ_per_serving(total_weight, model)
    
    meatbone = calculate_meatbone_per_serving(total_weight, model)
    meatbone_relevant = model['muscle meat'] + model['bone']
    relevant_meat_percent = model['muscle meat']/meatbone_relevant
    relevant_bone_percent = model['bone']/meatbone_relevant
    bone = meatbone * relevant_bone_percent
    meaty_bone = bone/df_bone_content.loc[df_bone_content.Type_of_Bone.str.lower() == bone_type.lower(), 'Bone_percentage'].values[0]
    muscle_meat = meatbone-meaty_bone
    
    ingredients = dict(**vfn_serving, **organ_serving)
    ingredients['muscle_meat'] = muscle_meat
    ingredients[bone_type] = meaty_bone
    
    return ingredients

def calculate_daily_serving(weight_kg, feed_weight_ratio, homemade_feed_ratio):
    return weight_kg * 1000 * weight_ratio * homemade_raw_percentage

In [57]:
daily_serving = calculate_daily_serving(weight_kg=8, 
                                        feed_weight_ratio=0.035, 
                                        homemade_feed_ratio=0.03)
print("Daily serving:", daily_serving)

Daily serving: 84.0


In [54]:
meal_prep(daily_serving, 1, puppy_barf_model, 'chicken neck')

{'veggie': 5.880000000000001,
 'fruit': 0.84,
 'nut': 2.52,
 'liver': 5.880000000000001,
 'other_organ': 5.880000000000001,
 'muscle_organ': 6.72,
 'muscle_meat': 39.480000000000004,
 'chicken neck': 16.8}